# Makine öğrenmesi 

In [1]:
def getAnalysis(score):
    if score < 0:
        return "-1"
    elif score > 0:
        return "+1"
    else:
        return "0"

In [2]:
import pandas as pd
data = pd.read_csv(r"kanalistanbul\kanalistanbul.csv")

data["Analysis"] = data["Polarity"].apply(getAnalysis)

data.drop(['Subjectivity', 'Polarity'], axis=1, inplace=True)

print(data.head())

                                               Tweet Analysis
0  next digit 195  resistors 199 218 250 301 352 ...       +1
1                                   kanalistanbul ;;        0
2  Kanalistanbul, but Nehiri?stanbul, necessary f...       -1
3  I think they are rehearsing the historical isl...       +1
4  really great change in istanbul i wish everywh...       +1


In [3]:
x = data["Tweet"]
y = data["Analysis"]

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(analyzer="word", lowercase=True)

sen_vector = vect.fit_transform(x.values.astype('U')) 
# You need to convert the dtype object to unicode string as is clearly mentioned in the traceback.

print(sen_vector)

  (0, 2642)	0.12737146643912098
  (0, 8144)	0.11049904498808594
  (0, 8715)	0.03977252991779308
  (0, 10720)	0.058352779357196266
  (0, 9123)	0.0780640652013316
  (0, 15030)	0.13093750135412802
  (0, 6346)	0.06773017282971103
  (0, 14332)	0.0747497342619574
  (0, 8112)	0.04340452034298359
  (0, 5076)	0.14942003360691017
  (0, 9516)	0.12178869809748634
  (0, 392)	0.17012204366244468
  (0, 1252)	0.08463979527343728
  (0, 296)	0.17776253961520694
  (0, 8411)	0.07762874133571425
  (0, 2836)	0.05155346035797346
  (0, 14956)	0.1541557478120368
  (0, 665)	0.13635851950413794
  (0, 12117)	0.12673263300267815
  (0, 12918)	0.2510329664082362
  (0, 14190)	0.1327806302949902
  (0, 9401)	0.11335964434356378
  (0, 1482)	0.1401884127569399
  (0, 144)	0.17776253961520694
  (0, 153)	0.17776253961520694
  :	:
  (9194, 14190)	0.19344819091142584
  (9194, 1482)	0.340400990394524
  (9195, 1308)	0.3018633190598801
  (9195, 645)	0.3018633190598801
  (9195, 213)	0.3018633190598801
  (9195, 11672)	0.3018633190

In [5]:
# train 
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(sen_vector,y,test_size=0.2,random_state=0)

## NAIVE BAYES



## -------------------------------------------------------

In [6]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X=x_train.toarray(), y=y_train)

GaussianNB()

In [7]:
print("score: ", gnb.score(x_test.toarray(),y_test))

score:  0.3760869565217391


## DECISION TREE

In [8]:
from sklearn.tree import DecisionTreeClassifier
clf =DecisionTreeClassifier()
clf.fit(X=x_train.toarray(), y=y_train)
print("score: ", gnb.score(x_test.toarray(),y_test))


score:  0.3760869565217391


## -------------------------------------------------------

In [9]:
from keras.utils import to_categorical
import numpy as  np

train_encoded = to_categorical(y_train)
test_encoded = to_categorical(y_test)

n_sent,n_vect = np.shape(x_train)
xx_train = np.reshape(x_train.toarray(),(n_sent,n_vect,1))

nn_sent,nn_vect = np.shape(x_test)
xx_test = np.reshape(x_test.toarray(),(nn_sent,nn_vect,1))

## CNN

In [11]:
#gerekli kütüphaneleri import ettik
from sklearn.utils import Bunch
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sn
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation


In [30]:
#Modellerimizi ekliyoruz
model = Sequential()
model.add(Conv1D(64, 2, activation="relu", input_shape=(2,1)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dense(16, activation="relu"))
model.add(Flatten())
model.add(Dense(4, activation = 'relu'))
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = "Adam", metrics = ['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1, 64)             192       
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 64)             256       
_________________________________________________________________
activation_1 (Activation)    (None, 1, 64)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1, 16)             1040      
_________________________________________________________________
flatten_1 (Flatten)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 68        
Total params: 1,556
Trainable params: 1,428
Non-trainable params: 128
__________________________________________________

In [31]:
import tensorflow as tf

model.fit(x_train,train_encoded, batch_size=16,epochs=400, verbose=0,validation_data=(x_test, test_encoded))

import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss=history.history['val_loss']
epochs = range(1, len(loss)+1)
plt.plot(epochs, loss, label='Eğitim Kayıp Değeri')
plt.plot(epochs, val_loss, label='Test Kayıp Değeri')
plt.title('Eğitim ve Test Kayıp Değerleri')
plt.xlabel('Deneme')
plt.ylabel('Kayıp')
plt.legend()
plt.show();
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Doğruluk Grafiği')
plt.ylabel('doğruluk')
plt.xlabel('deneme')
plt.legend(['train', 'test'], loc='upper right')
plt.show()
acc = model.evaluate(x_train, train_encoded)
print("Loss:", acc[0], " Accuracy:", acc[1])
pred = model.predict(x_test)
pred_y = pred.argmax(axis=-1)
cm = confusion_matrix(y_test, pred_y)
sn.heatmap(cm, annot=True)


InvalidArgumentError: indices[3] = [0,9875] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SerializeManySparse]

In [11]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [12]:
score = model.evaluate(x_test,y_test,verbose=0)
print("Loss", score[0])
print("acc", score[1])

ValueError: in user code:

    C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1233 test_function  *
        return step_function(self, iterator)
    C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1224 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\DELL\AppD

## RNN

In [14]:
import numpy as  np
import matplotlib.pyplot as plt
import pandas as pd



x = data.iloc[:, [0,-1]].values
y = data.iloc[:, 1].values


from sklearn.model_selection import train_test_split

x = x.reshape(x.shape[0], x.shape[1], 1)
x_train, x_test, y_train, y_test = train_test_split(sen_vector,y,test_size=0.8,random_state=0)

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))

x_test, y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))


from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout
from keras.regularizers import l2

regressor = Sequential()

regressor.add(SimpleRNN(units= 100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                        activation='tanh', use_bias = True, return_sequences = True, input_shape = (x_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units= 100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                        activation='tanh', use_bias = True, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units= 100, kernel_regularizer=l2(0.01), recurrent_regularizer = l2(0.01), bias_regularizer = l2(0.01),
                        activation='tanh', use_bias = True, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units= 100))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss ='binary_crossentropy', metrics=['accuracy'])


#RNN'i eğitme

training_model = regressor.fit(x_train, y_train, epochs = 10, batch_size = 32, validation_data = (x_test, y_test), verbose = 1)


predictions = regressor.predict(x_test)
score = regressor.evaluate(x_test,y_test)
print("\nLoss:",score[0])
print("\nAccuarcy:",score[1])

import matplotlib.pyplot as plt

print(training_model.history.keys())
# summarize history for accuracy
plt.plot(training_model.history['accuracy'])
plt.plot(training_model.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc ='upper left')
plt.show()
# summarize history for loss
plt.plot(training_model.history['loss'])
plt.plot(training_model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc ='upper left')
plt.show()

IndexError: tuple index out of range

## MLP

In [34]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
model = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=1000, activation='tanh', solver='adam', alpha=1e-5, learning_rate='constant',
                      verbose=1, early_stopping=True)
history=model.fit(x_train, train_encoded)
y_true, y_pred = test_encoded, model.predict(x_test)
#print(classification_report(y_true, y_pred))


loss = history.history['loss']
val_loss=history.history['val_loss']
epochs = range(1, len(loss)+1)
plt.plot(epochs, loss, label='Eğitim Kayıp Değeri')
plt.plot(epochs, val_loss, label='Test Kayıp Değeri')
plt.title('Eğitim ve Test Kayıp Değerleri')
plt.xlabel('Deneme')
plt.ylabel('Kayıp')
plt.legend()
plt.show();
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Doğruluk Grafiği')
plt.ylabel('doğruluk')
plt.xlabel('deneme')
plt.legend(['train', 'test'], loc='upper right')
plt.show()
acc = model.evaluate(x_train, train_encoded)
print("Loss:", acc[0], " Accuracy:", acc[1])
pred = model.predict(x_test)
pred_y = pred.argmax(axis=-1)
cm = confusion_matrix(test_encoded, pred_y)
sn.heatmap(cm, annot=True)

Iteration 1, loss = 1.29495444
Validation score: 0.747283
Iteration 2, loss = 0.71851470
Validation score: 0.830163
Iteration 3, loss = 0.26689807
Validation score: 0.842391
Iteration 4, loss = 0.09266316
Validation score: 0.861413
Iteration 5, loss = 0.04133489
Validation score: 0.860054
Iteration 6, loss = 0.02361328
Validation score: 0.851902
Iteration 7, loss = 0.01439579
Validation score: 0.857337
Iteration 8, loss = 0.01028251
Validation score: 0.860054
Iteration 9, loss = 0.00715866
Validation score: 0.868207
Iteration 10, loss = 0.00507559
Validation score: 0.865489
Iteration 11, loss = 0.00359674
Validation score: 0.860054
Iteration 12, loss = 0.00292961
Validation score: 0.862772
Iteration 13, loss = 0.00244244
Validation score: 0.860054
Iteration 14, loss = 0.00209618
Validation score: 0.862772
Iteration 15, loss = 0.00181611
Validation score: 0.865489
Iteration 16, loss = 0.00160612
Validation score: 0.865489
Iteration 17, loss = 0.00144138
Validation score: 0.860054
Iterat

AttributeError: 'MLPClassifier' object has no attribute 'history'

## LSTM

In [15]:
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
embed_dim = 128

lstm_out = 196

model = Sequential()


model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(2,activation='softmax'))

model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [18]:

model.fit(x_train,train_encoded, batch_size=16,epochs=400, verbose=0,validation_data=(x_test, test_encoded))

InvalidArgumentError: indices[3] = [0,9875] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SerializeManySparse]